
# Quiver Plots & Movies

In [ ]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

from pathlib import Path
import subprocess

import matplotlib.pyplot as plt
import cmocean
from tqdm.auto import tqdm

import dustpy
import dustpy.constants as c
from plot_helper import plot_quiver, read_paletton_text, plot_size_distri
import helper

# Read the paletton color file

colors = (read_paletton_text()[[3,1,2,0], 0, :]/255).tolist()
plt.imshow([colors]).axes.axis('off');

In [ ]:
reader = dustpy.hdf5writer()
data_dir = Path('~').expanduser() / 'DATA/araa_data/highres_old/data_bump'
reader.datadir = str(data_dir)
files = reader.read.listfiles()

In [ ]:
file = Path(files[-1])

## Make a PDF

In [ ]:
ax = plot_size_distri([str(file) for file in files], radii_au=[3, -30, 100], cols=colors)
ax.set_facecolor('none')
helper.output_dir
#f.savefig(helper.output_dir / 'size_distribution.pdf', transparent=True)

In [ ]:
output_file = helper.output_dir / file.with_suffix('.pdf').name.replace('data', 'frame_')

f, ax, sols = plot_quiver(str(file), vmin=1e-8, cmap='cmo.algae', cols=colors)
ax.set_facecolor('none')
f.savefig(output_file, transparent=True)

## Make the frames & compile movie

In [ ]:
dirname = Path('frames')
dirname.mkdir(exist_ok=True)

for i, file in tqdm(enumerate(files), total=len(files)):
    f, ax = plot_quiver(file, vmin=1e-7)
    
    f.savefig(dirname / f'frame_{i:03d}.jpg', transparent=False, dpi=300)

    #ax.set_facecolor('none')
    #f.savefig(dirname / f'frame_{i:03d}.png', transparent=True, dpi=300)
    
    plt.close(f)

Transparent

In [ ]:
ret = subprocess.check_output(
    f'ffmpeg -y -r 12 -i {dirname}/frame_%03d.png -c:v prores_ks -profile:v 5 -vendor apl0  -bits_per_mb 8000 -pix_fmt yuva444p10le out_t.mov'.split(),
    stderr=subprocess.STDOUT)

Non-Transparent

In [ ]:
ret = subprocess.check_output(
    f'ffmpeg -y -framerate 7 -i {dirname}/frame_%03d.png -c:v libx264 -crf 23 -pix_fmt yuv420p out.mp4'.split(),
    stderr=subprocess.STDOUT)